In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [3]:
def clean_figure(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis="x", rotation=45)

## What % of PC / LNC RNA / novel multiexonic transcripts do we remove w/ filtering?

In [11]:
mt_df = pd.read_csv('../data/05_mastertable/master_table.tsv', sep='\t')
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)

In [12]:
# limit to PC and lnc and novel
mt_df = mt_df.loc[(mt_df.associated_gene_biotype.isin(['lncRNA', 'protein_coding']))|\
                  (mt_df.associated_gene_biotype.isnull())]
n = len(mt_df.index)

In [16]:
# get things passing filtering
mt_df = pd.read_csv('../data/05_mastertable/poder_master_table_fixed_genics.tsv', sep='\t')
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)
n2 = len(mt_df.index)

In [19]:
print(f'We discard {n-n2}/{n} multiexonic PC / lnc / novel gene transcripts ({((n-n2)/n)*100})')

We discard 222472/378347 multiexonic PC / lnc / novel gene transcripts (58.801047715456974)


In [18]:
print(n2)

155875


## How many novel loci do we find ? (minus fusion)

In [21]:
# get things passing filtering
mt_df = pd.read_csv('../data/05_mastertable/poder_master_table_fixed_genics.tsv', sep='\t')
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)

# get novel only\
nov_structural_cats = ['Antisense', 'Intergenic', 'Genic']
mt_df = mt_df.loc[mt_df.structural_category.isin(nov_structural_cats)]

In [24]:
len(mt_df['geneid.v'].unique())

1214